In [ ]:
# Instalo bibliotecas
!pip install bs4 --quiet

In [ ]:
# Importo
import pandas as pd
import requests
import time
import concurrent.futures

from bs4 import BeautifulSoup

In [ ]:
# Inicializacion de constantes

# Defino cantidad de topicos y paginas
topics = ['economia', 'el-mundo', 'sociedad']
pages = range(1, 5)

# Defino headers del request
request_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}

In [ ]:
# Creo funcion que retorna las noticias de un topico determinado
def retrieve_topic_news(topic):
  topic_news = []
  for page in pages:
    request = requests.get(f'https://www.pagina12.com.ar/secciones/{topic}?page={page}', headers=request_headers)
    soup = BeautifulSoup(request.text, 'html.parser')
    articles = soup.find_all('article', class_='article-item article-item--teaser ')
    for article in articles:
      anchor = article.find('a', class_='p12-separator--left--primary')
      if anchor is not None:
        article_detail = requests.get(f'https://www.pagina12.com.ar/{anchor["href"]}', headers=request_headers)
        soup_detail = BeautifulSoup(article_detail.text, 'html.parser')
        paragraphs = soup_detail.find('div', class_='article-main-content article-text ').find_all('p')
        news = ''
        for paragraph in paragraphs:
          if paragraph.string is not None:
            news += paragraph.string
        topic_news.append(news)
    time.sleep(1)
  return {topic: topic_news}

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
  topics_results = executor.map(retrieve_topic_news, topics)

  topics_news = dict()
  for topic_result in topics_results:
    topics_news.update(topic_result)

In [ ]:
topics_news

{'economia': ['Durante la jornada, la empresa Marolio inauguró su nuevo centro de distribución de alimentos hacia todo el país, para el que destinaron una inversión de 580 millones de pesos, y la nueva línea de producción de pan rallado y rebozador, que demandó 130,4 millones de pesos. "Los empresarios nacionales saben que la caída de los salarios no es la solución, ya que se quedan sin demanda y la mejores épocas de crecimiento se dieron con Gobiernos que distribuyeron mejor los ingresos”, dijo el gobernador.\xa0“Hoy estamos saliendo de la pandemia con un esfuerzo enorme y, aunque debimos postergar algunos proyectos, nunca olvidamos que nuestros objetivos son cuidar la salud, la educación y la producción”, explicó Kicillof y concluyó: “Necesitamos una dirigencia empresarial dispuesta a desarrollar la industria bonaerense y a acompañarnos para generar más trabajo y producción en la Provincia, porque eso significa también más igualdad y más justicia social”.\n',
  '',
  'Uber anunció es